In [1]:
import numpy as np
import pandas as pd
import altair as alt
from typing import Literal

## 6

In [2]:
# each f output based on input x
# target function == f(x)
possibleTargetFuncOutputs = np.array([
    [0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]
])

# input
xOutsideD = np.array(
    [
        [1,0,1], [1,1,0], [1,1,1]
    ]
)

answerSelections = {
    "a": 0,
    "b": 0,
    "c": 0,
    "d": 0,
}

In [3]:
def fAndGNumAgreements(g, boolVectors: np.ndarray, possibleTargetFuncOutputs: np.ndarray, maxDisagreements: int):
    # add max agreements parameter?
    # for target fuc output
    # bothFucAgreed = False
    # maxDisagreements = ? 0, 1, 2, 3,
    # for output value
    # expectedAndHypotAgreed = False
    # x-vector outside of D input into g(x)
    # if g(x) == expected f(x) output value
    #   expectedAndHypotAgreed = True
    # else expectedAndHypotAgreed = False
    #  if not expectedAndHypotAgreed and maxDisagreements 0:
    #   break/return False maybe bothFucAgreed = False
    # 
    # 
    # outer loop if bothFucAgreed:
    #   numOfAgreementFunc += 1
    # f(x) already computed and is the matrix of target function outputs
    numOfAgreementFunc = 0
    for fucOutput in possibleTargetFuncOutputs:
        bothFucAgreed = False
        numOfAgreements = 0
        # currentDisagreements = 0
        for i, output in enumerate(fucOutput):
            bothFucAgreed = g(boolVectors[i]) == output
            if bothFucAgreed:
                numOfAgreements += 1
        if numOfAgreements == (3 - maxDisagreements):
            numOfAgreementFunc += 1
    
    return numOfAgreementFunc


def scoreG(g, boolVectors: np.ndarray, possibleTargetFuncOutputs: np.ndarray):
    finalScore = 0
    numPoints = boolVectors.ndim
    # loop 3 (i) times to score start @ 1? (skip the agreements with 0 since x * 0 = 0)
    # call fAndGNumAgreements with max disagreements of i
    # then finalScore += fAndGNumAgreements * i
    # return finalScore
    numOfDisagreements = 0
    while numOfDisagreements < 3:
        finalScore += fAndGNumAgreements(g, boolVectors, possibleTargetFuncOutputs, numOfDisagreements) * numOfDisagreements
        numOfDisagreements += 1
    return finalScore



In [4]:
# a
# returns 1 on all vectors
def gA(threeDBoolVector):
    return 1


print(scoreG(gA, xOutsideD, possibleTargetFuncOutputs))
answerSelections["a"] = scoreG(gA, xOutsideD, possibleTargetFuncOutputs)


9


In [5]:
# b
# returns 0 on all vectors
def gB(threeDBoolVector):
    return 0


print(scoreG(gB, xOutsideD, possibleTargetFuncOutputs))
answerSelections["b"] = scoreG(gB, xOutsideD, possibleTargetFuncOutputs)

9


In [6]:
# c
# XOR odd num of 1 = 1, even num  of 1 = 0
from functools import reduce
from operator import xor

# takes the XOR cumulatively starting with first 2 elements then XOR results with element 3.... and onwards
def gC(threeDBoolVector: np.ndarray):
    return reduce(xor, threeDBoolVector)

print(scoreG(gC, xOutsideD, possibleTargetFuncOutputs))
answerSelections["c"] = scoreG(gC, xOutsideD, possibleTargetFuncOutputs)

9


In [7]:
# d
# the opposite XOR odd num of 1 = 0, even num  of 1 = 1
from functools import reduce
from operator import xor

# the inverse of XOR is XOR so just flipping the bits to reverse the function provided in option d in the HW 1
#  1,0,1 => 0,1,0 thus returning the opposite of the XOR function in the end
#    0        1
def flipBit(bit: int):
    return bit ^ 1

# XOR elements cumulatively but flips the bits first to get the  opposite of the XOR
def gD(threeDBoolVector: np.ndarray):
    return reduce(xor, map(flipBit, threeDBoolVector))

print(scoreG(gD, xOutsideD, possibleTargetFuncOutputs))
answerSelections["d"] = scoreG(gD, xOutsideD, possibleTargetFuncOutputs)

9


#### [e] They are all equivalent (equal scores for g in [a] through [d]).
Correct

## The Perceptron Learning Algorithm

**Take-Aways/notes**:

In finding the disagreement probability between f & g, I approximated this value by generating a large dataset of separate random points.

check: numbers 9-10

#### The Perceptron Learning Algorithm (in 2 dimensions)
##### intial W = 0 vector to miss classify all points

In [8]:
class PLA:
    def __init__(self, x: np.ndarray, y: np.ndarray, w: np.ndarray = np.array([[0,0,0]], dtype="float64").T):
        ''' The Perceptron Learning Algorithm (in 2 dimensions)
        x: ndarray
            all inputs as column vectors in a matrix
            
            We also treat x[n], where n is a column, as a column vector and modify it to become
            x[n] = [x0, xi, · · · , xd]T, where the added coordinate x0 is fixed at x0 = 1.
        y: ndarray
            corresponding correct classification of a x[n], where n is a column
        w: ndarray
            a weight column vector

            we will treat the bias b
            as a weight wo = b and merge it with the other weights into one vector
            w = [w0, w1 , · · · , wd]T thus w is column vector

            defaults to the zero vector in 2-D if no intial weight vector is provided
        '''
        self.x = x
        self.y = y
        self.w = w
        # for Learning from data HW #1 assignment
        self.countWeightVectorUpdates = 0

    def incrementCountWeightVectorUpdates(func):
        def wrapper(self, *args, **kwargs):
            self.countWeightVectorUpdates += 1
            func(self, *args, **kwargs)
        return wrapper
    
    # @ is matrix multiplication
    def perceptronFormula(self, W: np.ndarray, X: np.ndarray) -> Literal[1,-1]:
        '''
        W: ndarray
            Weight vector as column vector
        X: ndarray
            input column vector

        h(x)/perceptron formula

        ### Returns
        y/class: ndarray
            +1 or -1: "where one side of the line maps to +1 and the other maps to -1." thus a binary classification???
        '''
        tempW = W
        return np.sign(W.transpose() @ X).item()
    
    @incrementCountWeightVectorUpdates
    def updateWeightVector(self, y: Literal[1,-1], X: np.ndarray):
        '''
        y: 1 | -1
            the correct??? classification of the x input vector

        x: ndarray
            an input column vector
        '''
        # y correct classification 
        # X vector that maps to the y classification
        self.w = self.w + (y * X).reshape((3,1))
        #                  ^ scalar
    
    def train(self):
        '''
        Trains The Perceptron Learning Algorithm model based on provided x, y and w values provided during instantiation
        '''
        column = 0
        # run until all points have been correctly classified
        while column != self.x.shape[1]:
            columnVector = self.x[:,column]
            # if not (classification := self.y[:,column]) == self.perceptronFormula(self.w, columnVector):
            # if current model is NOT able to correctly classify the point
            if (classification := self.y[:,column]) != self.perceptronFormula(self.w, columnVector):
                self.updateWeightVector(classification.item(), columnVector)
                # run start check from beginning again
                column = 0
            else:
                column += 1

    def predict(self):
        pass


In [9]:
def generatePLATestYs(w: np.ndarray, x: np.ndarray) -> np.ndarray:
    return np.sign(w.transpose() @ x)

#### Generate Random Line stuff

In [10]:
# points in R 2
def slopeOfTwoPoints(X: np.ndarray, Y: np.ndarray) -> float:
    return (Y[1] - Y[0])/(X[1] - X[0])

# points in R 2
def equationOfLineSlopInterceptForm(X: np.ndarray, Y: np.ndarray) -> dict[str, float]:
    '''
    y = mx + b
    '''
    line = {}
    line["slope"] = slopeOfTwoPoints(X, Y)
    line["yIntercept"] = -line["slope"] * X[0] + Y[0]
    return line


def generateRandomLine(numOfPoints: int, lowerBound: int, upperBound: int):
    # column 0 = x coordinates
    # column 1 = y coordinates (each nested array is a point)
    RandomPoints = np.random.uniform(lowerBound, upperBound, size=(numOfPoints,2))
    return (equationOfLineSlopInterceptForm(RandomPoints[:,0], RandomPoints[:,1]), RandomPoints)

# using vectorization for improved performance and practice
def calculateYsForLine(X: np.ndarray, line: dict[str, float]) -> np.ndarray:
    return (line["slope"] * X) + line["yIntercept"]


#### PLA Experiment function

In [11]:
def generate_random_x_y_data_set(numOfTrainingPoints: int, targetFInVectorForm: np.ndarray, dimension: int = 2, rngLowerBound: int = -1, rngUpperBound: int = 1):
    # Generate random X values to train the algorithm
    # generates some ground truth data using the target function f
    # that the algorithm will use to find a function g that approximates??? or equals f

    # might need to change this after intial testing
    # is it random if random is seeded from a fixed seed?
    randomState = np.random.RandomState(seed=0)

    # each row is a vector
    # thus columns are xi
    randomPointsVectors = randomState.uniform(rngLowerBound, rngUpperBound, size=(numOfTrainingPoints, dimension))

    '''
    We also treat x as a column vector and modify it to become x =
    [x0, xi, · · · , xd]T, where the added coordinate x0 is fixed at x0 = 1.

    '''
    x0 = np.ones(shape=randomPointsVectors.shape[0])

    # append rows of ones that will be x0 for each vector
    # make XTrain into column vectors in the matrix
    XTrain = np.column_stack((x0, randomPointsVectors)).T

    # YTrain are the classification the point belongs to (1 | -1)
    YTrain = generatePLATestYs(w=targetFInVectorForm, x=XTrain)
    return (XTrain, YTrain)

In [12]:
def approximate_f_and_g_Disagreement(f: np.ndarray, g: np.ndarray, numTestPoints: int):
    XTest, YTest = generate_random_x_y_data_set(numTestPoints, f)
    YPredict = generatePLATestYs(g, XTest)
    # np.array_equal(YPredict, YTest)
    disagreements = np.where(YPredict != YTest)
    
    disagreementsPercent = disagreements[0].shape[0] / numTestPoints

    return disagreementsPercent
    

In [13]:
def runPLAExperiment(numRuns: int, numTrainPoints: int, numTestPoints: int, pointsINRunLowerBound: int = -1, pointsINRunUpperBound: int = 1, dimension: int = 2):
    '''
    In order to get a reliable estimate repeat the
    experiment for 1000 runs

    Used to answer HW # 7 - 10

    returns: tuple[float, float]
                index 0: average number of iterations it took The Perceptron Learning Algorithm model (g) to converge to random target f
                index 1: average probability that f & g will disagree on their classification of a random point
    '''
    
    totalMaxCountWeightVectorUpdatesPerRun = 0
    disagreementsPercentPerRun = np.empty(numRuns)
    # for loop the below
    for iteration in range(numRuns):
        fLine, fLineOriginPoints = generateRandomLine(2, pointsINRunLowerBound, pointsINRunUpperBound)

        '''
        we will treat the bias b
        as a weight wo = b and merge it with the other weights into one vector
        w = [w0, w1 , · · · , wd]T

        We also treat x as a column vector and modify it to become x =
        [x0, xi, · · · , xd]T, where the added coordinate x0 is fixed at x0 = 1.

        '''
        # equation of a line in "standard form" = x + y + b = 0 used to make column vector
        # note "-" signs
        targetFInVectorForm = np.array([
            [
                -fLine["yIntercept"], # aka b
                -fLine["slope"], # x coefficient
                1 # y coefficient
            ]
        ])

        # make vector a column vector
        targetFInVectorForm = targetFInVectorForm.T
        XTrain, YTrain = generate_random_x_y_data_set(numTrainPoints, targetFInVectorForm)

        # test PLA and train g to approximate f or equal
        plaModel = PLA(x=XTrain, y=YTrain)
        plaModel.train()
        g = plaModel.w

        # used for mean calculations to answer HW questions
        disagreementsPercentPerRun[iteration] = approximate_f_and_g_Disagreement(targetFInVectorForm, g, numTestPoints)
        totalMaxCountWeightVectorUpdatesPerRun += plaModel.countWeightVectorUpdates

    meanConvergence = totalMaxCountWeightVectorUpdatesPerRun / numRuns
    meanDisagreementsPercent = np.mean(disagreementsPercentPerRun)

    return (meanConvergence, meanDisagreementsPercent)

#### Main Program

In [14]:
# runPLAExperiment(1000, 10, -1, 1, 2)

In [15]:
# create target function
dimensions = 2
targetFOriginalNumPoints = 2
fPointLowerBound = -1
fPointUpperBound = 1
# XsVector = np.array([3,2])
# YsVector = np.array([5,6])

# line = equationOfLineSlopInterceptForm(XsVector, YsVector)

line, lineOriginPoints = generateRandomLine(targetFOriginalNumPoints, fPointLowerBound, fPointUpperBound)

'''
we will treat the bias b
as a weight wo = b and merge it with the other weights into one vector
w = [w0, w1 , · · · , wd]T

We also treat x as a column vector and modify it to become x =
[x0, xi, · · · , xd]T, where the added coordinate x0 is fixed at x0 = 1.

'''
# equation of a line in "standard form" = x + y + b = 0 used to make column vector
# note "-" signs
targetFInVectorForm = np.array([
    [
        -line["yIntercept"], # aka b
        -line["slope"], # x coefficient
        1 # y coefficient
    ]
])

# make vector a column vector
targetFInVectorForm = targetFInVectorForm.T
# targetFInVectorForm.T

targetFInVectorForm.shape


(3, 1)

In [16]:
# Generate random X values to train the algorithm
# generates some ground truth data using the target function f
# that the algorithm will use to find a function g that approximates??? or equals f

rngLowerBound = -1
rngUpperBound = 1
numOfTrainingPoints = 200

randomState = np.random.RandomState(seed=0)

# each row is a vector
# thus columns are xi
randomPointsVectors = randomState.uniform(rngLowerBound, rngUpperBound, size=(numOfTrainingPoints, dimensions))

'''
We also treat x as a column vector and modify it to become x =
[x0, xi, · · · , xd]T, where the added coordinate x0 is fixed at x0 = 1.

'''
x0 = np.ones(shape=randomPointsVectors.shape[0])

# append rows of ones that will be x0 for each vector
# make XTrain into column vectors in the matrix
XTrain = np.column_stack((x0, randomPointsVectors)).T

# YTrain are the classification the point belongs to (1 | -1)
YTrain = generatePLATestYs(w=targetFInVectorForm, x=XTrain)
# points = n

In [17]:
# test PLA and get g to approximate f or equal
plaModel = PLA(x=XTrain, y=YTrain)
plaModel.train()
g = plaModel.w
plaModel.countWeightVectorUpdates

167

#### Plots

In [18]:
source = np.row_stack((XTrain, YTrain)).T
fLineColor = "#015220"
gLineColor = "#8b0000"


# each row is a vector
df = pd.DataFrame(source, columns=["x0", "x1", "x2", "class"])

# plot target F & hypothesis G function line: need to make dummy x data...
plotXs = np.arange(-100, 100)

# calculate y values for plotXs
dfFLine = pd.DataFrame(
    {
        "x": plotXs,
        # aka y/target function
        "f(x)": calculateYsForLine(plotXs, line)
    }
)

# transform vector that is equation of a line in standard form to
# slope intercept form y = mx + b
# thus needed to solve for y by divining by the y's coefficient
gLine = {
    "slope": -(g[1][0]/g[2][0]),
    "yIntercept": -(g[0][0]/g[2][0])
}

dfGLine = pd.DataFrame(
    {
        "x": plotXs,
        # aka g/hypothesis function
        "g(x)": calculateYsForLine(plotXs, gLine)
    }
)


In [19]:
points = alt.Chart(df.copy()).mark_circle(size=50).encode(
    x="x1:Q",
    y="x2:Q",
    color= alt.Color("class", scale=alt.Scale(domain=[-1,1], range=["red", "green"])),
    tooltip=[alt.Tooltip("x1:Q", title="x"), alt.Tooltip("x2:Q", title="y")]
).properties(
    height=900,
    width=900
).interactive()

linePlot = alt.Chart(dfFLine.copy()).mark_line().encode(
    x="x",
    y="f(x)",
    color=alt.value(fLineColor)
)

# plot g hypothesis hopefully approximates f well...

gLinePlot = alt.Chart(dfGLine.copy()).mark_line().encode(
    x="x",
    y="g(x)",
    color=alt.value(gLineColor)
)

points + linePlot + gLinePlot

alt.LayerChart(...)

#### 7: my HW Answer = b (I was correct)

In [20]:
# runPLAExperiment(1000, 10, 1000, -1, 1, 2)
# average in a run was 13.722
# |13.722 - 15| = 1.278 this is the closest to zero out of the 5 choices


#### 8: my HW Answer = c (I was correct)

In [21]:
# runPLAExperiment(1000, 10, 1000, -1, 1, 2)
# the probability that f and g will disagree on their classification of a random point = 0.114
# |0.114 - 0.1| = 0.014 this is the closest to zero out of the 5 choices

#### 9: my HW Answer = b (I was correct)

In [22]:
# runPLAExperiment(1000, 100, 1000, -1, 1, 2)
# average in a run was 251.491
# |251.491 - 100| = 1.278 this is the closest to zero out of the 5 choices

'''
The more data point we have in a set the longer the model has to train.
'''

#### 10: my HW Answer = b (I was correct)

In [23]:
# runPLAExperiment(1000, 100, 1000, -1, 1, 2)
# the probability that f and g will disagree on their classification of a random point = 0.114
# |0.011802000000000003 - 0.01| = 0.001802000000000003 this is the closest to zero out of the 5 choices

'''
this makes sense because the more data points used to train the model, g, the better g approximates f.
This means that there is less of a chance that g will make an "error".
'''

(183.664, 0.011802000000000003)